Imports

In [1]:
import neo
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats


Loadning of unprocessed data

In [23]:
# List of filenames for each eye
filenames = [
    "/Users/marielunde/Desktop/Thesis/all data/3D/Eye 1/10Hz_1V_neg500mV_1ms003.ns5",
    "/Users/marielunde/Desktop/Thesis/all data/3D/Eye 2/10Hz_1V_neg500mV_1ms002.ns5",
    "/Users/marielunde/Desktop/Thesis/all data/3D/Eye 3/10Hz_1V_neg500mV_1ms004.ns5",
    "/Users/marielunde/Desktop/Thesis/all data/3D/Eye 4/10Hz_1V_neg500mV_1ms018.ns5",
    "/Users/marielunde/Desktop/Thesis/all data/3D/Eye 5/10Hz_1V_neg500mV_1ms004.ns5",
    "/Users/marielunde/Desktop/Thesis/all data/3D/Eye 6/10Hz_1V_neg500mV_1ms011.ns5"
]

# Initialize an empty list to hold the data from all eyes
all_data = []

# Loop through each file and read the data
for filename in filenames:
    reader = neo.io.BlackrockIO(filename=filename)
    
    # Read the data from the file
    block_SA = reader.read_block()
    
    # Access the first segment
    segment_SA = block_SA.segments[0]
    
    # Access the first analog signal within this segment
    analog_signal_SA = segment_SA.analogsignals[0]
    
    # Access the signal data as a NumPy array and append it to the list
    data_SA = np.asarray(analog_signal_SA.magnitude)
    all_data.append(data_SA)

# Print shape of the data
print(f"Data shape: {np.shape(all_data)}")


Data shape: (6, 300300, 32)


NB the following functions also count the SA as a spike 

In [31]:
def find_spike_indices(eye_data, processed = False):
    """
    Function to find the indices of spikes in each channel of the given eye_data.

    Parameters:
        eye_data (pd.Series): A series containing the data for each channel.

    Returns:
        pd.DataFrame: A DataFrame where each column corresponds to a channel and
                      contains the list of spike indices (section, index) for that channel.
    """

    spike_indices = []  # List to store spike indices for each channel

    # Iterate over each channel
    for channel_idx, channel in enumerate(eye_data):
        channel_spike_indices = []  # List to store spike indices for the current channel

        if processed == False:
            channel = channel[:,np.newaxis]

        for section_idx, section in enumerate(channel.T[:, :]):  # Transpose to iterate over columns
            # Compute the threshold for the current section
            threshold = np.sqrt(np.mean(np.square(section), axis=0)) * 4.5

            detention = 0
            first_spike = True  # Flag to handle the first spike

            # Iterate over each data point in the section
            for i, data_point in enumerate(section):
                if first_spike:
                    if data_point > threshold:
                        # Look at the next 30 data points or until the end of the section
                        look_ahead_end = min(i + 30, len(section))
                        look_ahead_data = section[i:look_ahead_end]

                        # Find the index of the maximum value in the look-ahead window
                        max_index = np.argmax(look_ahead_data) + i  # Add i to get the correct index in the original section

                        # Append the index of the maximum value
                        channel_spike_indices.append((i, max_index))

                        first_spike = False
                        detention = 0
                else:
                    if data_point > threshold and detention > 30:
                        # Look at the next 30 data points or until the end of the section
                        look_ahead_end = min(i + 30, len(section))
                        look_ahead_data = section[i:look_ahead_end]

                        # Find the index of the maximum value in the look-ahead window
                        max_index = np.argmax(look_ahead_data) + i  # Add i to get the correct index in the original section

                        # Append the index of the maximum value
                        channel_spike_indices.append((i, max_index))

                        detention = 0
                    else:
                        detention += 1

        # Store the spike indices for the current channel
        spike_indices.append(channel_spike_indices)


    return spike_indices

In [36]:
def plot_spikes_with_data_unprocessed(eye_data, channel_idx, spike_indices, num_points=None):
    # Extract the data for the specific channel and section
    channel = eye_data[channel_idx]
    channel = channel[:, np.newaxis]
    section = channel.T[:]  # Extract the specific section data

    # Extract the spike indices for the specific section
    spike_points = [tup[1] for tup in spike_indices[channel_idx]]

    # If num_points is specified, limit the section and spike points
    if num_points is not None:
        section = section[:num_points]
        spike_points = [sp for sp in spike_points if sp < num_points]

    # Plot the data series
    plt.figure(figsize=(10, 6))
    plt.plot(section, label=f'Channel {channel_idx + 1}', color='blue')

    # Overlay the spikes with red dots
    plt.scatter(spike_points, section[spike_points], color='red', label='Spikes', zorder=5)

    # Set plot labels and title
    plt.xlabel('Data Point Index')
    plt.ylabel('Amplitude')
    plt.title(f'Unprocessed Data Series with Spikes for Channel {channel_idx + 1}')

    # Add legend
    plt.legend()

    # Show the plot
    plt.grid(True)
    plt.show()


In [42]:
spike_indices = find_spike_indices(all_data[0].T, processed = False)
print(len(spike_indices))
print(f"Shape of spike_indices: {[len(indices) for indices in spike_indices]}")
# plot_spikes_with_data_unprocessed(all_data[0].T, 0, spike_indices, num_points=3000)

32
Shape of spike_indices: [216, 253, 345, 287, 221, 238, 234, 229, 264, 249, 233, 259, 239, 274, 225, 249, 325, 228, 237, 346, 246, 231, 265, 328, 290, 243, 251, 0, 222, 280, 266, 259]
